In [1]:
import pickle
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter

import time
import datetime

print(datetime.datetime.now())

2022-08-09 14:01:01.374070


In [2]:
path_data = 'data/d20220201/'

# INPUTS
fn_finalSave = path_data + 'featuresAndMeta_s20220720.p'
fn_predictionsTraining  = path_data + 'predictionsTraining_binary_s20220704.xlsx'
fn_predictionsValidation = path_data + 'predictionsValidation_binary_s20220704.xlsx'
fn_predictionsTraining_3g  = path_data + 'predictionsTraining_s20220302.xlsx'
fn_predictionsValidation_3g = path_data + 'predictionsValidation_s20220519.xlsx'


plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [3]:
with open(fn_finalSave, "rb") as input_file:
    dict_data = pickle.load(input_file)

dict_data.keys()

dict_keys(['df_rawFeatures_training', 'df_rawFeatures_validation', 'dict_normalizationFactors', 'df_zscoreFeatures_training', 'df_zscoreFeatures_validation', 'df_metadata', 'umap_raw', 'umap_zscore'])

In [4]:
predictions_train = pd.read_excel(fn_predictionsTraining, index_col=0)
predictions_validations = pd.read_excel(fn_predictionsValidation, index_col=0)
predictions_binary = pd.concat([predictions_train, predictions_validations])
predictions_binary['id'] = predictions_binary.index
predictions_binary['id'] = predictions_binary['id'].apply(lambda x: str(x).zfill(4))
predictions_binary = predictions_binary.set_index('id')

predictions_train = pd.read_excel(fn_predictionsTraining_3g, index_col=0)
predictions_validations = pd.read_excel(fn_predictionsValidation_3g, index_col=0)
predictions_3g = pd.concat([predictions_train, predictions_validations])
predictions_3g['id'] = predictions_3g.index
predictions_3g['id'] = predictions_3g['id'].apply(lambda x: str(x).zfill(4))
predictions_3g = predictions_3g.set_index('id')

In [5]:
currMeta = dict_data['df_metadata']
currMeta.isnull().sum()

trainTest           0
age                 0
sex                 0
ct_grp              0
lre               120
decompBL          120
lrd               120
lrd_tf            120
survival          120
survival_tf       120
tfs               120
tfs_tf            120
hvpg              120
hvpg_corrected    120
hvpg_grp          120
hepLrd_time       120
hepLrd_180        120
hepLrd_360        120
cps               120
cps_grp           120
meld              120
meld_grp          120
etiology          120
sodium            120
creatinine        120
albumin           488
CRP               120
INR               120
bilirubin         120
patientID           0
dtype: int64

In [ ]:
currMeta = currMeta.dropna()

# make variables
temp = pd.get_dummies(currMeta['meld_grp'], prefix='meldgrp')
currMeta = pd.concat([currMeta, temp], axis = 1)
temp = pd.get_dummies(currMeta['cps_grp'], prefix='cpsgrp')
currMeta = pd.concat([currMeta, temp], axis = 1)

currMeta['etio_alcLiverDisease'] = currMeta['etiology'].apply(lambda x: True if x == '1' else False)
currMeta['etio_nonAlcFattyLiver'] = currMeta['etiology'].apply(lambda x: True if x == '2' else False)
currMeta['etio_viral'] = currMeta['etiology'].apply(lambda x: True if x == '3' else False)
currMeta['etio_other'] = currMeta['etiology'].apply(lambda x: True if x == '4' else False)
currMeta['etio_unknown'] = currMeta['etiology'].apply(lambda x: True if x == '5' else False)

# add radipop predictions
currMeta = currMeta.merge(predictions_binary[['grp']], how='inner', left_index=True, right_index=True)
currMeta = currMeta.rename(columns={"grp": "pred_bin"})
currMeta = currMeta.merge(predictions_3g[['grp']], how='inner', left_index=True, right_index=True)
currMeta = currMeta.rename(columns={"grp": "pred_3g"})

currMeta

In [7]:
display(pd.DataFrame(data={'type': currMeta.dtypes, 'nans': currMeta.isnull().sum()}))

,type,nans
trainTest,object,0
age,int64,0
sex,object,0
ct_grp,object,0
lre,float64,0
decompBL,object,0
lrd,float64,0
lrd_tf,object,0
survival,float64,0
survival_tf,object,0


In [8]:
cov_sets = {'cps': ['age', 'sex',
                    'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
                    'cps_grp',
                    'sodium', 'creatinine'],
            'meld': ['age', 'sex',
                     'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
                     'meld',
                     'albumin'],
            'radipop': ['age', 'sex',
                        'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
                        'sodium', 'creatinine', 'CRP', 'bilirubin', 'albumin', 'INR', 
                        'pred_bin'],
            'radipop+cps': ['age', 'sex',
                            'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
                            'cps_grp',
                            'sodium', 'creatinine',
                            'pred_bin'],
            'radipop+meld': ['age', 'sex',
                             'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
                             'meld',
                             'albumin',
                             'pred_bin'],            
            'measured': ['age', 'sex',
                         'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
                         'sodium', 'creatinine', 'CRP', 'bilirubin', 'albumin', 'INR', 
                         'hvpg_corrected']
           }

In [9]:
# # baseline compensated ('decompBL'=False)
# subset_comp = currMeta[currMeta['decompBL']==False]

# timetuples = [('survival', 'survival_tf'),
#               ('lre', 'decompBL')]

# for eaTuple in timetuples:
#     time = currMeta[eaTuple[0]]
#     status = currMeta[eaTuple[1]]
#     kmf = KaplanMeierFitter()
#     kmf.fit(durations = time, event_observed = status)

#     fig, ax = plt.subplots(figsize=(8, 6))
#     kmf.plot_survival_function()
#     plt.title(eaTuple[0])

#     temp = currMeta[[eaTuple[0], eaTuple[1], 
#                      'age', 'sex',
#                      'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other', #'etio_unknown',
#                      'cps', #'cpsgrp_1', 'cpsgrp_2', 'cpsgrp_3',
#                      'meld', #'meldgrp_1', 'meldgrp_2',
#                      'sodium', 'creatinine', 'CRP']]
#     cph = CoxPHFitter()
#     cph.fit(temp, duration_col = eaTuple[0], event_col = eaTuple[1])
#     cph.print_summary()

In [10]:
subset_comp = currMeta[currMeta['decompBL']==False]

timetuples = [('survival', 'survival_tf'),]
#              ('lre', 'decompBL')] ##could not be run, high Ícollinearity

allModels={}
results = pd.DataFrame()

for eaTuple in timetuples:
    for eaCovset in cov_sets:
        curr_cov = copy.deepcopy(cov_sets[eaCovset])
        curr_cov.extend([eaTuple[0], eaTuple[1]])
        temp = subset_comp[curr_cov]
        cph = CoxPHFitter()
        cph.fit(temp, duration_col = eaTuple[0], event_col = eaTuple[1])
        #cph.print_summary()
        modelname = eaTuple[0] + '_' + eaCovset
        allModels[modelname] = cph
        
        currResults = dict(cph.hazard_ratios_)
        currResults['AIC'] = cph.AIC_partial_
        currResults['Concordance'] = cph.concordance_index_
        curr_df = pd.DataFrame.from_dict(currResults, 
                                         orient='index',
                                         columns=[modelname])
        results = pd.merge(results, curr_df, how='outer', left_index=True, right_index=True)

/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1122: ConvergenceWarning: Column cps_grp have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['survival_tf'].astype(bool)
>>> print(df.loc[events, 'cps_grp'].var())
>>> print(df.loc[~events, 'cps_grp'].var())

A very low variance means that the column cps_grp completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Convergen

In [11]:
# reorder rows
orderIndex = ['age', 'sex',
              'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
              'sodium', 'creatinine', 'CRP', 'INR', 'bilirubin', 'albumin',
              'cps_grp', 'meld',
              'pred_bin', 'hvpg_corrected',
              'AIC', 'Concordance'
             ]
results = results.reindex(orderIndex)
results

,survival_cps,survival_meld,survival_radipop,survival_radipop+cps,survival_radipop+meld,survival_measured
age,9.807864e-01,1.271540e+00,0.982100,9.756456e-01,0.992341,0.984751
sex,1.913478e-02,3.331667e-03,0.387324,2.455246e-02,0.163614,0.487991
etio_alcLiverDisease,2.603179e+07,1.066109e+06,2.582213,1.923028e+07,6.694990,2.304686
etio_nonAlcFattyLiver,6.221163e+08,3.004863e+12,11.303791,5.283180e+08,3084.916582,8.876058
etio_viral,2.023256e+07,6.856787e+11,6.787141,2.021715e+07,1144.720402,8.304189
etio_other,5.344001e+07,3.092816e+08,3.698145,4.857374e+07,245.390891,4.439682
sodium,1.428071e+00,NaN,1.173497,1.359306e+00,NaN,1.041854
creatinine,2.640407e+03,NaN,6.985650,1.696116e+03,NaN,3.158759
CRP,NaN,NaN,4.093072,NaN,NaN,2.494503
INR,NaN,NaN,91.862068,NaN,NaN,12.980201


In [12]:
results.iloc[:,0:6]

,survival_cps,survival_meld,survival_radipop,survival_radipop+cps,survival_radipop+meld,survival_measured
age,9.807864e-01,1.271540e+00,0.982100,9.756456e-01,0.992341,0.984751
sex,1.913478e-02,3.331667e-03,0.387324,2.455246e-02,0.163614,0.487991
etio_alcLiverDisease,2.603179e+07,1.066109e+06,2.582213,1.923028e+07,6.694990,2.304686
etio_nonAlcFattyLiver,6.221163e+08,3.004863e+12,11.303791,5.283180e+08,3084.916582,8.876058
etio_viral,2.023256e+07,6.856787e+11,6.787141,2.021715e+07,1144.720402,8.304189
etio_other,5.344001e+07,3.092816e+08,3.698145,4.857374e+07,245.390891,4.439682
sodium,1.428071e+00,NaN,1.173497,1.359306e+00,NaN,1.041854
creatinine,2.640407e+03,NaN,6.985650,1.696116e+03,NaN,3.158759
CRP,NaN,NaN,4.093072,NaN,NaN,2.494503
INR,NaN,NaN,91.862068,NaN,NaN,12.980201


In [13]:
results.iloc[:,6:12]

""
age
sex
etio_alcLiverDisease
etio_nonAlcFattyLiver
etio_viral
etio_other
sodium
creatinine
CRP
INR


In [14]:
# # baseline decompensated ('decompBL'=True)
# subset_decomp = currMeta[currMeta['decompBL']==True]
# timetuples = [('survival', 'survival_tf')]

# for eaTuple in timetuples:
#     time = currMeta[eaTuple[0]]
#     status = currMeta[eaTuple[1]]
#     kmf = KaplanMeierFitter()
#     kmf.fit(durations = time, event_observed = status)
    
#     fig, ax = plt.subplots(figsize=(8, 6))
#     kmf.plot_survival_function()
#     plt.title(eaTuple[0])

# for eaTuple in timetuples:
#     temp = currMeta[[eaTuple[0], eaTuple[1], 
#                      'age', 'sex',
#                      'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other', #'etio_unknown',
#                      'cps', #'cpsgrp_1', 'cpsgrp_2', 'cpsgrp_3',
#                      'meld', #'meldgrp_1', 'meldgrp_2',
#                      'sodium', 'creatinine', 'CRP']]
#     cph = CoxPHFitter()
#     cph.fit(temp, duration_col = eaTuple[0], event_col = eaTuple[1])
#     cph.print_summary()

In [15]:
# baseline decompensated ('decompBL'=True)
subset_decomp = currMeta[currMeta['decompBL']==True]
timetuples = [('survival', 'survival_tf')]


allModels={}
results = pd.DataFrame()

for eaTuple in timetuples:
    for eaCovset in cov_sets:
        curr_cov = copy.deepcopy(cov_sets[eaCovset])
        curr_cov.extend([eaTuple[0], eaTuple[1]])
        temp = subset_decomp[curr_cov]
        cph = CoxPHFitter()
        cph.fit(temp, duration_col = eaTuple[0], event_col = eaTuple[1])
        #cph.print_summary()
        modelname = eaTuple[0] + '_' + eaCovset
        allModels[modelname] = cph
        
        currResults = dict(cph.hazard_ratios_)
        currResults['AIC'] = cph.AIC_partial_
        currResults['Concordance'] = cph.concordance_index_
        curr_df = pd.DataFrame.from_dict(currResults, 
                                         orient='index',
                                         columns=[modelname])
        results = pd.merge(results, curr_df, how='outer', left_index=True, right_index=True)

/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:1607: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.804. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  exceptions.ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/pyt

In [16]:
# reorder rows
orderIndex = ['age', 'sex',
              'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
              'sodium', 'creatinine', 'CRP', 'INR', 'bilirubin', 'albumin',
              'cps_grp', 'meld',
              'pred_bin', 'hvpg_corrected',
              'AIC', 'Concordance'
             ]
results = results.reindex(orderIndex)
display(results)

,survival_cps,survival_meld,survival_radipop,survival_radipop+cps,survival_radipop+meld,survival_measured
age,1.027511e+00,1.028648e+00,1.041139,1.026477e+00,1.028857e+00,1.046695
sex,4.883431e-01,6.354233e-01,0.465367,5.763035e-01,6.818362e-01,0.446430
etio_alcLiverDisease,1.910482e+06,1.376615e+06,517501.069814,1.897770e+06,1.317232e+06,836755.268619
etio_nonAlcFattyLiver,7.006898e+05,6.352592e+05,169022.242372,7.907207e+05,6.263218e+05,232208.104268
etio_viral,1.311393e+06,1.142848e+06,385434.710548,1.318673e+06,1.079121e+06,483766.547800
etio_other,6.452898e+05,5.933851e+05,330533.703585,1.058669e+06,7.378557e+05,248053.711488
sodium,1.026918e+00,NaN,1.055626,1.021162e+00,NaN,1.065964
creatinine,2.004003e+00,NaN,5.497270,2.161026e+00,NaN,5.831978
CRP,NaN,NaN,0.640844,NaN,NaN,0.630403
INR,NaN,NaN,5.768355,NaN,NaN,5.114825


In [17]:
##
# baseline compensated ('decompBL'=False)
subset_comp = currMeta[currMeta['decompBL']==False]

timetuples = [('survival', 'survival_tf'),]
#              ('lre', 'decompBL')]

# univariable
allCov = ['age', 'sex',
          'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
          'sodium', 'creatinine', 'CRP', 'bilirubin', 'albumin', 'INR', 
          'cps_grp', 'meld',
          'pred_bin', 'hvpg_corrected']

uniResults = {}
uni_df = pd.DataFrame()

for eaTuple in timetuples:
    for eaCov in allCov:
        temp = subset_comp[[eaCov, eaTuple[0], eaTuple[1]]]
        cph = CoxPHFitter()
        cph.fit(temp, duration_col = eaTuple[0], event_col = eaTuple[1])
        
        uniResults[eaCov] = cph.hazard_ratios_[0]
    curr_df = pd.DataFrame.from_dict(uniResults,
                                     orient='index',
                                     columns=[eaTuple[0]])
    uni_df = pd.merge(uni_df, curr_df, how='outer', left_index=True, right_index=True)

/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__i

In [18]:
uni_df

,survival
age,0.918893
sex,0.277449
etio_alcLiverDisease,0.726327
etio_nonAlcFattyLiver,2.449720
etio_viral,1.032213
etio_other,1.235258
sodium,1.478760
creatinine,0.460067
CRP,3.029294
bilirubin,2.413712


In [19]:
##
# baseline decompensated ('decompBL'=True)
subset_decomp = currMeta[currMeta['decompBL']==True]
timetuples = [('survival', 'survival_tf')]

# univariable
allCov = ['age', 'sex',
          'etio_alcLiverDisease', 'etio_nonAlcFattyLiver', 'etio_viral', 'etio_other',
          'sodium', 'creatinine', 'CRP', 'bilirubin', 'albumin', 'INR', 
          'cps_grp', 'meld',
          'pred_bin', 'hvpg_corrected']

uniResults = {}
uni_df = pd.DataFrame()

for eaTuple in timetuples:
    for eaCov in allCov:
        temp = subset_decomp[[eaCov, eaTuple[0], eaTuple[1]]]
        cph = CoxPHFitter()
        cph.fit(temp, duration_col = eaTuple[0], event_col = eaTuple[1])
        
        uniResults[eaCov] = cph.hazard_ratios_[0]
    curr_df = pd.DataFrame.from_dict(uniResults,
                                     orient='index',
                                     columns=[eaTuple[0]])
    uni_df = pd.merge(uni_df, curr_df, how='outer', left_index=True, right_index=True)

/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__init__.py:1187: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)
/opt/conda/lib/python3.7/site-packages/lifelines/utils/__i

In [20]:
uni_df

,survival
age,1.023499
sex,0.650183
etio_alcLiverDisease,2.032425
etio_nonAlcFattyLiver,1.077603
etio_viral,0.661035
etio_other,0.444622
sodium,0.988237
creatinine,2.336381
CRP,0.793929
bilirubin,1.074237


In [21]:
print(datetime.datetime.now())

2022-08-09 14:01:07.117556
